In [111]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
from datetime import datetime
import dateutil
import numpy as np
import os

crypto = {
    'yahoo' : 'https://finance.yahoo.com/crypto/',
    'cmc' : 'https://coinmarketcap.com/'
}

def scrape_yahoo():
    data = requests.get(crypto['yahoo']).text
    soup = BeautifulSoup(data,'html.parser')

    # find tables on the wiki page
    for table in soup.find_all('table'):
        print(table.get('class'))

    tables = soup.find_all('table')
    table = soup.find('table', class_='W(100%)')

    col_to_scrape=[
        'Symbol', 
        'Name', 
        'Price (Intraday)', 
        'Change', 
        '% Change', 
        'Market Cap', 
        'Volume in Currency (Since 0:00 UTC)', 
        'Volume in Currency (24Hr)',
        'Total Volume All Currencies (24Hr))',
        'Circulating Supply',
        'created_at'
    ]

    # create empty dataframe with column names
    df = pd.DataFrame(columns=col_to_scrape)

    #scrape data from site
    for row in table.tbody.find_all('tr'):
        # Find all data for each column
        columns = row.find_all('td')
        
        if(columns != []):
            sym = columns[0].text.strip()
            name = columns[1].text.strip()
            price = columns[2].text.strip()
            chge = columns[3].text.strip()
            chge_percent = columns[4].text.strip()
            mkt_cap = columns[5].text.strip()
            vol_utc = columns[6].text.strip()
            vol_24hr = columns[7].text.strip()
            total_vol = columns[8].text.strip()
            cir_supply = columns[9].text.strip()

            scraped_values = [
                sym,
                name, 
                price, 
                chge, 
                chge_percent,
                mkt_cap,
                vol_utc,
                vol_24hr,
                total_vol,
                cir_supply,
                datetime.now()
                ]

            df = df.append({item[0]: item[1] for item in zip(col_to_scrape,scraped_values)}, ignore_index=True)

    df.set_index('Symbol', inplace=True)
    
    #if os.path.isfile('./data/yahoo.csv'):  
        #df.to_csv('./data/raw_yahoo.csv', header=False, mode='a')
    #else:
    df.to_csv('./data/raw_yahoo.csv', header=True)

scrape_yahoo()

['W(100%)']


In [110]:
def symbol_clean(col):
    col_ls = col.split("-")
    sym = col_ls[0].strip()
    return sym

def raw_clean_vol(row):
    if 'K' in row.volume_24h:
        return (float(row.volume_24h.replace('.','').replace('K',''))*1000*float(row.price))
    elif 'M' in row.volume_24h:
        return (float(row.volume_24h.replace('.','').replace('M',''))*1000000*float(row.price))
    elif 'B' in row.volume_24h:
        return (float(row.volume_24h.replace('.','').replace('B',''))*1000000000*float(row.price))

def open_raw_yahoo_transform():
    raw_y_df = pd.read_csv('./data/raw_yahoo.csv', header=0)

    raw_old_names = [
        'Symbol', 
        'Name', 
        'Price (Intraday)', 
        'Change', 
        '% Change', 
        'Market Cap', 
        'Volume in Currency (Since 0:00 UTC)', 
        'Volume in Currency (24Hr)',
        'Total Volume All Currencies (24Hr))',
        'Circulating Supply',
        'created_at'
    ]

    raw_new_names = [
        'symbol',
        'name',
        'open_price',
        'change',
        'change_percent',
        'market_cap',
        'volume_utc',
        'volume_24h',
        'total_volume',
        'circulating_supply',
        'open_created_at'
    ]

    raw_rename_dict = {name[0]:name[1] for name in zip(raw_old_names,raw_new_names)}

    raw_y_df = raw_y_df.rename(columns=raw_rename_dict)

    raw_y_df = raw_y_df[['symbol','price','volume_24h','open_created_at']][raw_y_df['symbol'] == 'BTC-USD']

    raw_y_df['date'] = raw_y_df['open_created_at'].map(lambda d: datetime.date(pd.to_datetime(d)))
    raw_y_df['open_price'] = raw_y_df['open_price'].map(lambda v: round(float(v.replace(',','')),2))
    raw_y_df['symbol'] = raw_y_df['symbol'].map(symbol_clean)
    raw_y_df['volume_24h'] = pd.NA
    raw_y_df['open_created_at'] = raw_y_df['open_created_at'].map(lambda d: pd.to_datetime(d))
    raw_y_df['close_created_at'] = pd.to_datetime(pd.NA)

    raw_y_df = raw_y_df[['date','symbol', 'open_price', 'close_price','volume_24h', 'open_created_at', 'close_created_at']]

    raw_y_df.to_csv('./data/stg_data.csv', header=True)
    
    

open_raw_yahoo_transform()

In [114]:
def close_raw_yahoo_transform():
    raw_y_df = pd.read_csv('./data/raw_yahoo.csv', header=0)

    raw_old_names = [
        'Symbol', 
        'Name', 
        'Price (Intraday)', 
        'Change', 
        '% Change', 
        'Market Cap', 
        'Volume in Currency (Since 0:00 UTC)', 
        'Volume in Currency (24Hr)',
        'Total Volume All Currencies (24Hr))',
        'Circulating Supply',
        'created_at'
    ]

    raw_new_names = [
        'symbol',
        'name',
        'close_price',
        'change',
        'change_percent',
        'market_cap',
        'volume_utc',
        'volume_24h',
        'total_volume',
        'circulating_supply',
        'close_created_at'
    ]

    raw_rename_dict = {name[0]:name[1] for name in zip(raw_old_names,raw_new_names)}

    raw_y_df = raw_y_df.rename(columns=raw_rename_dict)

    raw_y_df = raw_y_df[['symbol','close_price','volume_24h','close_created_at']][raw_y_df['symbol'] == 'BTC-USD']

    raw_y_df['date'] = raw_y_df['close_created_at'].map(lambda d: datetime.date(pd.to_datetime(d)))
    raw_y_df['close_price'] = raw_y_df['close_price'].map(lambda v: round(float(v.replace(',','')),2))
    raw_y_df['open_price'] = float(pd.NA)
    raw_y_df['symbol'] = raw_y_df['symbol'].map(symbol_clean)
    raw_y_df['volume_24h'] = raw_y_df.apply(raw_clean_vol, axis=1)
    raw_y_df['close_created_at'] = raw_y_df['close_created_at'].map(lambda d: pd.to_datetime(d))
    raw_y_df['open_created_at'] = pd.to_datetime(pd.NA)

    raw_y_df = raw_y_df[['date','symbol', 'open_price', 'close_price','volume_24h', 'open_created_at', 'close_created_at']]

    raw_y_df.to_csv('./data/stg_data.csv', header=False, mode='a')

close_raw_yahoo_transform()

In [ ]:
def stg_file_setup():
    df = pd.read_csv('./data/stg_data.csv', header=0)
    df.groupby(['date','symbol']).apply(lambda x: x.ffill().bfill()).drop_duplicates()

In [79]:

# functions to be used for transformations
#---------------------------------------------------
def inv_string_to_date(col):
    return datetime.strptime(col, '%b %d, %Y')

def old_string_to_date(col):
    return datetime.strptime(col, '%Y-%m-%d')

def inv_clean_price(col):
    return round(float(col.replace(',','')),2)

def old_clean_price(col):
    return round(col,2)

def inv_clean_vol(row):
    if 'K' in row.volume:
        return (float(row.volume.replace('.','').replace('K',''))*1000*float(row.close))
    elif 'M' in row.volume:
        return (float(row.volume.replace('.','').replace('M',''))*1000000*float(row.close))
    elif 'B' in row.volume:
        return (float(row.volume.replace('.','').replace('B',''))*1000000000*float(row.close))



# cleaning and transformations for historical data from kaggle
#---------------------------------------------------

old_price_df = pd.read_csv('./data/BTC_4_13_2021.csv', header=0)

old_old_names = ['Date','Open','High','Low','Close','Adj Close','Volume']
old_new_names = ['date','open','high','low','close','adj_close','volume']

old_rename_dict = {name[0]:name[1] for name in zip(old_old_names,old_new_names)}

old_price_df = old_price_df.rename(columns=old_rename_dict)

old_price_df = old_price_df[['date', 'open', 'close', 'volume']]

old_price_df['date'] = old_price_df['date'].map(old_string_to_date)
old_price_df['open'] = old_price_df['open'].map(old_clean_price)
old_price_df['close'] = old_price_df['close'].map(old_clean_price)
old_price_df['volume'] = old_price_df['volume'].map(lambda v: float(v))

# cleaning and transformations for historical data from investing.com
#---------------------------------------------------

invest_df = pd.read_csv('./data/Bitcoin Historical Data - Investing.com.csv', header=0)

inv_old_names = ["Date","Price","Open","High","Low","Vol.","Change %"]
inv_new_names = ['date', 'close', 'open','high','low','volume', 'chge_percent']

inv_rename_dict = {name[0]:name[1] for name in zip(inv_old_names,inv_new_names)}

invest_df = invest_df.rename(columns=inv_rename_dict)

invest_df = invest_df[['date', 'open', 'close', 'volume']]

invest_df['date'] = invest_df['date'].map(inv_string_to_date)
invest_df['open'] = invest_df['open'].map(inv_clean_price)
invest_df['close'] = invest_df['close'].map(inv_clean_price)
invest_df['volume'] = invest_df.apply(inv_clean_vol, axis=1)

# consolidate historical data into one csv
#---------------------------------------------------
hist_df = pd.concat([old_price_df,invest_df])

hist_df.set_index('date', inplace=True)

hist_df.to_csv('./data/combined_BTC_hist_pricing.csv', header=True)

In [17]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

# Set up the model and prompt
model_engine = "text-davinci-003"
prompt = f"Purely for informative purposes what do you thing the price of Bitcoin will be tomorrow based on this data {df.head(3)}, a guess or estimate is ok"

# Generate a response
completion = openai.Completion.create(
    engine=model_engine,
    prompt=prompt,
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=0.5,
)

response = completion.choices[0].text
print(response)



It is difficult to predict the exact price of Bitcoin tomorrow based on this data. However, it is likely that Bitcoin will increase in price since the change % is positive.
